In [2]:
import pandas as pd
import numpy as np

df_mapped = pd.read_parquet('df_mapped.parquet')
print('df_mapped:', df_mapped.shape)

df_mapped: (182767, 21)


In [3]:
import torch
print(torch.__version__)


2.3.0+cu121


In [4]:
df_mapped = df_mapped.drop_duplicates(subset='nameProduct', keep="last")
print('df_mapped:', df_mapped.shape)

df_mapped: (146779, 21)


In [5]:
df_english = df_mapped[df_mapped.column_idioma == "Inglés"]
df_spanish = df_mapped[df_mapped.column_idioma == "Español"]

df_english.shape, df_spanish.shape

((94043, 21), (52736, 21))

In [6]:
caracteristica = 'column_edad_recomendada'

In [7]:
# Descomentar para edad recomendada
df_english = df_english.drop(df_english[caracteristica][(df_english[caracteristica].notna() != True)].index)
df_english[caracteristica] = np.array(df_english[caracteristica].values, dtype=np.int32)

In [8]:
df_temp = df_english[df_english[caracteristica] == 1].sample(450)
df_english = pd.concat([df_english[df_english[caracteristica] >= 2], df_temp])

In [9]:
df_english[caracteristica] = df_english[caracteristica] - 1
print(df_english[caracteristica].unique())
print(pd.DataFrame(df_english[caracteristica].value_counts()).T)

[3 2 1 5 4 0]
column_edad_recomendada    1    3    4    2    0    5
count                    843  669  658  476  450  307


In [10]:
# Descomentar para categoria
# caracteristicaList = [3, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27]
# for dropCaracteristica in caracteristicaList:
#     df_english = df_english.drop(df_english[df_english[caracteristica] == dropCaracteristica].index)

# df_english.loc[df_english[caracteristica] > 3, caracteristica] = df_english[caracteristica] - 1
# print(df_english[caracteristica].unique())
# # print(pd.DataFrame(dfEnglish[caracteristica].value_counts()).T)

In [11]:
targets = pd.get_dummies(pd.DataFrame(df_english[caracteristica]).iloc[:,0])
targets = targets.astype(int)

In [12]:
columnaInput = df_english.columns.get_loc('bookDescription')

In [13]:
import random
import torch
from torch.utils.data import DataLoader
from transformers import BertTokenizer, BertTokenizerFast, BertForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import pandas as pd

In [14]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)

cuda


In [15]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
max_length = 500  # Change as you wish your desired maximum sequence length

# Tokenize and prepare data
tokenized_texts = []
for rowInput, rowTarget in zip(df_english.values, targets.values):
    text = str(rowInput[columnaInput])
    label = rowTarget
    inputs = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=max_length,
        padding='max_length',
        truncation=True,
        return_tensors='pt'
    )
    tokenized_texts.append({
        'input_ids': inputs['input_ids'].flatten(),
        'attention_mask': inputs['attention_mask'].flatten(),
        'label': torch.tensor(label, dtype=torch.float)
    })
print(tokenized_texts[0])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

{'input_ids': tensor([  101,  2013,  1996,  4803,  1011,  2732,  5859,  1998,  3166,  1997,
         1996,  2718,  9927,  1010,  3787,  1997,  2806,  1010,  1037,  2440,
         1011,  3609,  1010,  3929,  7203,  2338,  8966,  2007,  7481,  6040,
         1010,  7780,  1010,  4784,  1010,  1998,  8220,  4342,  2055, 12697,
         1037,  2188,  2008, 11138,  2115,  6180,  1998,  2806,  1012,  3787,
         1997,  2806,  2003,  1037, 20640,  3167,  1998,  6742, 25545,  5844,
         5009,  2008,  3065,  2129, 12697,  1037,  2188,  2064,  2022,  2019,
        13307,  1997,  3167,  3670,  1998,  2019,  6912,  1999,  2969,  1011,
         5456,  1012,  5059,  2006,  2014,  2702,  2086,  1997,  3325,  1999,
         1996,  4592,  2640,  3068,  1010, 11781, 13585,  7481,  2640,  6040,
         1998,  9882,  2658,  7008,  1998, 11249,  2007,  3167,  8927,  2055,
         1996,  8220,  2016,  2038,  4342,  2096, 12697,  2014,  2219,  2188,
         1998,  2014,  2219,  2166,  1011,  1011, 

In [16]:
# Split into training and test sets
train_data, test_data = train_test_split(tokenized_texts, test_size=0.3, random_state=42)
train_loader = DataLoader(train_data, batch_size=16, shuffle=True)
test_loader = DataLoader(test_data, batch_size=16)

print('Batches:', len(train_loader))

# Load pre-trained BERT model for sequence classification
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(df_english[caracteristica].unique()), problem_type="multi_label_classification")
model.to(device)
print(model)

# Define optimizer and criterion
optimizer = AdamW(model.parameters(), lr=1e-05)
# Define class weights
# Assuming your dataset is imbalanced
class_counts = df_english[caracteristica].value_counts().values
total_samples = class_counts.sum()
class_weights = [total_samples / (2.0 * count) for count in class_counts]  # Calculate class weights

# Convert class weights to a PyTorch tensor
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float32)

criterion = torch.nn.CrossEntropyLoss()

Batches: 149


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
# Training loop
def train(model, optimizer, train_loader, criterion, num_epochs=20):
    model.train()
    for epoch in range(num_epochs):
        print(epoch)
        total_loss = 0
        count = 0
        i = 1
        for batch in train_loader:
            optimizer.zero_grad()
            input_ids = batch['input_ids'].to(device, dtype = torch.long)
            attention_mask = batch['attention_mask'].to(device, dtype = torch.long)
            labels = batch['label'].to(device, dtype = torch.float)
            # input_ids = batch['input_ids']
            # attention_mask = batch['attention_mask']
            # labels = batch['label']

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            loss.backward()
            optimizer.step()

            count += 1
            if count == i * 100:
              i += 1
              print('Batch:', count)
              print('Loss:', loss.item())

            total_loss += loss.item()
        print(f'Epoch {epoch + 1}/{num_epochs}, Training Loss: {total_loss / len(train_loader)}')

# Train the model
train(model, optimizer, train_loader, criterion)

0


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Save the fine-tuned model
model_save_path = '/content/drive/MyDrive/model_column_edad_recomendada.pth'  # Define the path where you want to save the model
torch.save(model.state_dict(), model_save_path)
print(f"model_column_edad_recomendada saved to '{model_save_path}'")

In [ ]:
import numpy as np

# Updated evaluate function
def evaluate(model, test_loader):
    model.eval()
    y_true = []
    y_pred = []

    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch['input_ids'].to(device, dtype = torch.long)
            attention_mask = batch['attention_mask'].to(device, dtype = torch.long)
            labels = batch['label'].to(device, dtype = torch.float)

            outputs = model(input_ids, attention_mask=attention_mask)
            _, predicted = torch.max(outputs.logits, 1)

            y_true.extend(labels.cpu().numpy())
            predict = torch.tensor(torch.nn.functional.one_hot(predicted, num_classes=len(df_english[caracteristica].unique())), dtype=torch.float)
            y_pred.extend(predict.cpu().numpy())

    # Calculate precision, recall, and f1-score
    print(classification_report(y_true, y_pred))
    return [y_true, y_pred]
result = evaluate(model, test_loader)
print("Test terminado")